# Using an LLM to interact with Pandas

It is possible to use an LLM to interact with a Pandas dataframe, something that can be useful when encountering a large, new dataset with unfamiliar content. Basically, the scenario can be thought of as a combination of a simple [RAG][1] system combined with [tool-calling][2]. As the tool-calls are fairly complex, we'll make use of the [`llamaindex` framework][3] to do the [heavy lifting (docs)][4].

**Caveat**: This is a bleeding edge technique and thus error prone, _and_ it uses python's `eval` function to execute code written by the LLM on your behalf which is a potential security risk. **Consider yourself warned.**

[1]: ../RAG-tutorial/intro.ipynb
[2]: ../LLM-tool-calling/LLM-tool-calling.ipynb
[3]: https://www.llamaindex.ai
[4]: https://docs.llamaindex.ai/en/stable/api_reference/query_engine/pandas/

## Install prerequisites

In [ ]:
!pip -q install ollama llama-index llama-index-experimental  llama-index-llms-ollama

## A simple Pandas agent

### The big picture

The process will look something like the following figure (compare to the [RAG setup][1])

![Query pipeline](img/pipeline.png)


1. The user inputs a query which is transformed into a first prompt, adding information about the Pandas dataframe to target before feeding the prompt to the LLM.
2. The LLM processes the first prompt and make [_tool calls_][2] to Pandas to produce intermediary output (containing code, data and further instructions).
3. The output is combined with the original query to form a second prompt.
4. The second prompt is fed to the LLM to generate an answer to the user query.

<!-- Looks like this: input query_str -> pandas_prompt -> llm1 -> pandas_output_parser -> response_synthesis_prompt -> llm2

Additional connections to response_synthesis_prompt: llm1 -> pandas_instructions, and pandas_output_parser -> pandas_output. -->

[1]: ../RAG-tutorial/intro.ipynb
[2]: ../LLM-tool-calling/LLM-tool-calling.ipynb

As an example, we will use the [`titanic` dataset](https://jkarakas.github.io/Exploratory-Analysis-of-the-Titanic-Dataset/Titanic_Dataset_Exploratory_Analysis_No_Code.html) in `data/titanic.csv`.

First, import the necessary modules from `llama_index`:

In [ ]:
from llama_index.core.query_pipeline import (QueryPipeline as QP, Link, InputComponent)
from llama_index.experimental.query_engine.pandas import PandasInstructionParser
from llama_index.llms.ollama import Ollama
from llama_index.core import Settings
from llama_index.core import PromptTemplate

then import Pandas and load the dataset:

In [ ]:
import pandas as pd

df = pd.read_csv("data/titanic.csv")

In [ ]:
df.info()

In [ ]:
df.head()

## LLM instructions (prompts)

To get a better understanding of what goes on under the hood, let's look at the prompt _templates_ used:

In [ ]:
from llama_index.experimental.query_engine import PandasQueryEngine
query_engine = PandasQueryEngine(df=df, verbose=True)
prompts = query_engine.get_prompts()
for key in prompts.keys():
    print(key)
    print(prompts[key].template)

First, we have the general instructions:

```python
instruction_str = (
    "1. Convert the query to executable Python code using Pandas.\n"
    "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
    "3. The code should represent a solution to the query.\n"
    "4. PRINT ONLY THE EXPRESSION.\n"
    "5. Do not quote the expression.\n"
)

```

Then the template that combines 1) information about the dataframe (`df.head()`), 2) the general instructions, and 3) the user query:

```python
pandas_prompt_str = (
    "You are working with a pandas dataframe in Python.\n"
    "The name of the dataframe is `df`.\n"
    "This is the result of `print(df.head())`:\n"
    "{df_str}\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Expression:"
)
```


After making the tool calls, the code generated and used in the previous step, the resulting output, and the user query is combined into a final prompt:

```python
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)
```

With that short explanation, let's try it out:

In [ ]:
# prompt(1) in figure
pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, df_str=df.head(5)
)

pandas_output_parser = PandasInstructionParser(df)

# prompt(2) in figure
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)

# N.B. at this poinnt, the templates are not fully evaluated
# You can evaluate and print prompt(1), but prompt(2) requires output from first LLM processing step
print(pandas_prompt)

## Build Query Pipeline

<!-- Looks like this: input query_str -> pandas_prompt -> llm1 -> pandas_output_parser -> response_synthesis_prompt -> llm2

Additional connections to response_synthesis_prompt: llm1 -> pandas_instructions, and pandas_output_parser -> pandas_output. -->

In [ ]:

OLLAMA_HOST = 'http://10.129.20.4:9090'
OLLAMA_MODEL = 'qwen2.5-coder:latest' # 'deepseek-coder-v2:latest' # 'deepseek-r1:70b' # 'llama3.3:latest'

Settings.llm = Ollama(model=OLLAMA_MODEL, base_url=OLLAMA_HOST)

llm = Ollama(model=OLLAMA_MODEL, base_url=OLLAMA_HOST)

# N.B. It is possible to use different LLMs for the first and secon processing step, but we'll use the smae model for both steps (`llm1` and `llm2` below)
query_pipeline = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": llm,
    },
    verbose=True,
)
query_pipeline.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])
query_pipeline.add_links(
    [
        Link("input", "response_synthesis_prompt", dest_key="query_str"),
        Link(
            "llm1", "response_synthesis_prompt", dest_key="pandas_instructions"
        ),
        Link(
            "pandas_output_parser",
            "response_synthesis_prompt",
            dest_key="pandas_output",
        ),
    ]
)
# add link from response synthesis prompt to llm2
query_pipeline.add_link("response_synthesis_prompt", "llm2")

In [ ]:
response = query_pipeline.run(
    query_str="What is the correlation between survival and age?",
)

In [ ]:
print(response.message.content)

In [ ]:
print(df.head())

In [ ]:
response = query_pipeline.run(
    query_str="What are the keys of the columns?",
)

In [ ]:
print(response.message.content)

In [ ]:
response = query_pipeline.run(
    query_str="What is the key of the column outlining survial?",
)

In [ ]:
print(response.message.content)

In [ ]:
response = query_pipeline.run(
    query_str="Generate python code to plot survival rate versus fare using matplotlib. Choose an appropriate binsize. Show plot as well as code.",
)

In [ ]:
print(response.message.content)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming df is your DataFrame and it contains 'fare' and 'survived' columns
df['fare'] = pd.to_numeric(df['fare'], errors='coerce')  # Ensure fare is numeric
df = df.dropna(subset=['fare', 'survived'])  # Drop rows with missing values

# Group by fare and calculate the mean survival rate
survival_rate_by_fare = df.groupby('fare')['survived'].mean()

# Plot the survival rate versus fare
plt.figure(figsize=(10, 6))
plt.bar(survival_rate_by_fare.index, survival_rate_by_fare.values, color='blue')
plt.xlabel('Fare')
plt.ylabel('Survival Rate')
plt.title('Survival Rate vs Fare')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()